In [ ]:
#Danny Hong
#ECE472 - Deep Learning
#Assignment 4 Part 1: Classifying CIFAR-10

import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, MaxPool2D, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

#Reading in batch data for the training sets for the CIFAR-10 dataset:
def get_batches(data_batch_files):
  for i in range(len(data_batch_files)):
    with open(data_batch_files[i], 'rb') as file:
      batch = pickle.load(file, encoding='bytes')
      if i == 0:
        x_train = (batch[b'data'].reshape((len(batch[b'data']), 3, 32, 32)).transpose(0, 2, 3, 1)).astype('float32')
        y_train = batch[b'labels']
      else:
        x_train_temp = (batch[b'data'].reshape((len(batch[b'data']), 3, 32, 32)).transpose(0, 2, 3, 1)).astype('float32')
        y_train_temp = batch[b'labels']
        x_train = np.concatenate((x_train, x_train_temp), axis=0)
        y_train = np.concatenate((y_train, y_train_temp), axis=0)

  return x_train, y_train

#Reading in the files for the CIFAR-10 dataset
def get_data(data_file):
  with open(data_file, 'rb') as file:
    batch = pickle.load(file, encoding='bytes')
    x_data = (batch[b'data'].reshape((len(batch[b'data']), 3, 32, 32)).transpose(0, 2, 3, 1)).astype('float32')
    y_data = batch[b'labels']

  return x_data, y_data

#Normalizing the x data: 
def normalize_x_data(x_train, x_test):
  eps = 1e-7
  mean = np.mean(x_train,axis = (0, 1, 2, 3))
  std = np.std(x_train,axis = (0, 1, 2, 3))
  x_train = (x_train - mean)/(std + eps)
  x_test = (x_test - mean)/(std + eps)

  return x_train, x_test

#Decaying the learning rate decay over time
def learning_rate_schedule(epoch):
  learning_rate = 0.001
  if epoch > 75:
    learning_rate = 0.0005 
  if epoch > 100:
    learning_rate = 0.0003 
  if epoch > 125:
    learning_rate = 0.0001
  if epoch > 150:
    learning_rate = 0.00005
  if epoch > 175:
    learning_rate = 0.00003
  if epoch > 200:
    learning_rate = 0.00001

  return learning_rate

num_classes = 10
num_rows = 32
num_columns = 32
weight_decay = 0.001
batch_size = 512
epochs = 225

def main():
  x_train, y_train = get_batches(data_batch_files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5'])
  x_test, y_test = get_data(data_file = 'test_batch')

  x_train, x_test = x_train / 255, x_test / 255
  x_train, x_test = normalize_x_data(x_train, x_test)
    
  y_train, y_test = tf.keras.utils.to_categorical(y_train), tf.keras.utils.to_categorical(y_test)
    
  original_train_size = len(x_train)
  new_train_size = len(x_train) - len(x_test)
  x_val, y_val = x_train[new_train_size:original_train_size], y_train[new_train_size:original_train_size]
  x_train, y_train = x_train[0:new_train_size], y_train[0:new_train_size]

  #Preparing to fit the model with data augmentation
  data_generator = ImageDataGenerator(rotation_range = 15, width_shift_range = 0.1, height_shift_range = 0.1, horizontal_flip = True) 
  train_generator = data_generator.flow(x_train, y_train, batch_size)
  steps_per_epoch = len(x_train) // batch_size


  model = Sequential()

  model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = tf.nn.relu, padding = 'same', kernel_regularizer = l2(weight_decay), input_shape = (num_rows, num_columns, 3)))
  model.add(BatchNormalization())
  model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = tf.nn.relu, padding = 'same', kernel_regularizer = l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size = (2, 2)))
  model.add(Dropout(0.1))

  model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = tf.nn.relu, padding = 'same', kernel_regularizer = l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = tf.nn.relu, padding = 'same', kernel_regularizer = l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size = (2, 2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = tf.nn.relu, padding = 'same', kernel_regularizer = l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = tf.nn.relu, padding = 'same', kernel_regularizer = l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size = (2, 2)))
  model.add(Dropout(0.3))

  model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = tf.nn.relu, padding = 'same', kernel_regularizer = l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = tf.nn.relu, padding = 'same', kernel_regularizer = l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size = (2, 2)))
  model.add(Dropout(0.4))

  model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = tf.nn.relu, padding = 'same', kernel_regularizer = l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = tf.nn.relu, padding = 'same', kernel_regularizer = l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size = (2, 2)))
  model.add(Dropout(0.5))

  model.add(Flatten())
    
  model.add(Dense(num_classes, activation = tf.nn.softmax))
    
  model.summary()
    
  optimizer = RMSprop(learning_rate = 0.001, decay = 1e-6)
    
  model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

  model.fit(train_generator, validation_data = (x_val, y_val), steps_per_epoch = steps_per_epoch, epochs = epochs, verbose = 1, callbacks = [LearningRateScheduler(learning_rate_schedule)])
    
  score = model.evaluate(x_test, y_test)
  print('\nTest loss:', score[0])
  print('Test accuracy:', score[1])
    
main()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_10 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_11 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 64)       